In [ ]:
!pip install pytorch_lightning
import pytorch_lightning as pl
pl.__version__

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 40.0 MB/s eta 0:00:00


'2.4.0'

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, models
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.metrics import average_precision_score, roc_auc_score
from torch.utils.data import DataLoader
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import glob

In [ ]:
weights_url = "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth"
!wget -nc {weights_url}

--2024-10-24 12:37:24--  https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth
Resolving download.pytorch.org (download.pytorch.org)... 18.165.116.121, 18.165.116.36, 18.165.116.45, ...
Connecting to download.pytorch.org (download.pytorch.org)|18.165.116.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21444401 (20M) [binary/octet-stream]
Saving to: ‘efficientnet_b0_rwightman-7f5810bc.pth’

efficientnet_b0_rwi 100%[===================>]  20.45M   108MB/s    in 0.2s    

2024-10-24 12:37:24 (108 MB/s) - ‘efficientnet_b0_rwightman-7f5810bc.pth’ saved [21444401/21444401]



EfficientNet

In [ ]:
class EfficientNetB0(nn.Module):
  def __init__(self, list_of_classes, weights_path=None):
    super(EfficientNetB0, self).__init__()
    self.list_of_classes = list_of_classes
    self.num_of_classes = len(list_of_classes)
    self.model = self.load_efficientnet(weights_path)
    self.model.classifier = nn.Sequential(
        nn.Dropout(0.2, inplace=True),
        nn.Linear(in_features=1280, out_features=self.num_classes, bias=True),
        nn.Sigmoid
    )
  def foward(self, x):
    x = self.model(x)
    return x

  def load_efficientnet(self,weights_path):
    model = models.efficientnet_b0(weights=None)
    if weights_path:
      print("Loading weights")
      state_dict = torch.load(weights_path, weights_only=True)
      model.load_state_dict(state_dict)
    return model

In [ ]:
epochs = 20
labels = ['iscrowd', 'notcrowd']
batch_size = 32
efficientnet_weights_path = "efficientnet_b0_rwightman-7f5810bc.pth"
scheduler_step = 7
gamma = 0.1


Classifier

In [ ]:
class Classifier(pl.LightningModule):
  def __init__(self, list_of_labels, weights_path=None):
    super(Classifier, self).__init__()
    self.list_of_labels = list_of_labels
    self.num_classes = len(list_of_labels)
    self.model = EfficientNetB0(list_of_labels, efficientnet_weights_path)
    self.model.classifier = nn.Sequential(
        nn.Dropout(0.2, inplace=True),
        nn.Linear(in_features=1280, out_features=self.num_classes, bias=True),
        nn.Sigmoid
    )

    self.loss_fn = nn.BCELoss()

  def foward(self, x):
    return self.model(x)

  def training_step(self, batch):
    image, label = batch
    image = image.float()
    label = label.to(torch.float)
    pred = self(image)
    loss = self.loss_fn(pred, label)

    self.log('train_loss', loss, prog_bar=True)

    return loss

  def validation_step(self, batch):
    image, label = batch
    image = image.float()
    label = label.to(torch.float)
    pred = self(image)
    loss = self.loss_fn(pred, label)

    self.log('val_loss', loss, prog_bar=True)

    return loss

  # def image_by_count(pred, threshold):
  #   labels = []

  #   for pred in pred:
  #     num_objects = len(pred['boxes'])
  #     if num_objects >= threshold:
  #       labels.append("crowd")
  #     else:
  #       labels.append("not_crowd")
  #   return labels


  def conf_optim(self):
    optimizer = optim.Adam(self.parameteters(), lr = 0.001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= scheduler_step, gamma=gamma)
    return [optimizer], [scheduler]

In [ ]:
def Augmentations(data, batch_size, epochs):
  transform = transform.Compose([
      tranform.Resize((224,224)),
      transfrom.ToTensor(),
      transform.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])


image_path = ''

train_paths, val_paths = train_test_split(image_path, test_size=0.2, random_state=42)

train_dataset =

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

model = Classifier(labels, weights_path=efficientnet_weights_path)
def train_model():



